# 2. Create NetCDF Module

# 2.1 Import Functions

In [ ]:
import os
import glob
#import WAPORWA modules
os.chdir(r'C:\WA_Souss_Massa_Training\WAPOR\modules') #change to modules path
import WaPOR
import WA
from WA.pickle_basin import pickle_in,pickle_out
from WA.create_NC import main as make_netcdf
from WA.create_NC import _get_lats_lons 
import gdal
import tempfile

# 2.2 Generate Basin Object

In [ ]:
INPUT_FOLDER=r"C:\WA_Souss_Massa_Training\WAPOR\Souss_Massa\Input"
BASIN={'Name':'Souss_massa',   
        'Dir':   r"C:\WA_Souss_Massa_Training\WAPOR\Souss_Massa\Main",
        'time_range':['2012-01-01','2013-12-31'],
        'end_month':'DEC',        
        'geo_data':{
                   'basin':r"C:\WA_Souss_Massa_Training\WAPOR\Souss_Massa\shapefile\sous_massa_wgs.shp",
                      },
        'global_data':{
                     'grace':r"C:\WA_Souss_Massa_Training\WAPOR\Souss_Massa\Global\GRACE\GSFC.glb.200301_201607_v02.4-ICE6G",
                     'grand':r"C:\WA_Souss_Massa_Training\WAPOR\Souss_Massa\Global\GRanD\GRanD_reservoirs_v1_3.shp",
                     'wdpa':r"C:\WA_Souss_Massa_Training\WAPOR\Souss_Massa\Global\WDPA\WDPA_CatIandII_17countries.shp",
                     },
        'input_data':{
                      'yearly':{
                              'lcc':[r""+os.path.join(INPUT_FOLDER,'L2_LCC_A'),
                                     '-','Landcover Class'],
                                'lu':[r"C:\WA_Souss_Massa_Training\WAPOR\Souss_Massa\Main\data\luwa",
                                     '-','WA+ Landuse Categories'],
                              },
                      'monthly':{
                              'p':[r""+os.path.join(INPUT_FOLDER,'L1_PCP_M'),
                                   'mm/month','Precipitation'],
                              'et':[r""+os.path.join(INPUT_FOLDER,'L2_AETI_M'),
                                    'mm/month','Actual Evapotranspiration'],
                              'ret':[r""+os.path.join(INPUT_FOLDER,'L1_RET_M'),
                                    'mm/month','Reference Evapotranspiration'],
                              'i':[r""+os.path.join(INPUT_FOLDER,'L2_I_M'),
                                    'mm/month','Interception'],
                              'nRD':[r""+os.path.join(INPUT_FOLDER,'Rainy_Days'),
                                    'days/month','Number of Rainy Days'],
                              },  
                    'stat':{
                             'thetasat': [r"C:\WA_Souss_Massa_Training\WAPOR\Souss_Massa\Input\ThetaSat\SMsat_2013.06.01.tif",
                                          '%','Top-soil Saturated Water Content']
                            }                                                   
                              },
        'input_ts':{
                'dS':None,
                'Qoutlet':r"C:\WA_Souss_Massa_Training\WAPOR\Souss_Massa\Input\Souss_q_2012_2013.csv",
                'Qswout':None,
                'Qgwout':None,
                
                    },
        'main_data':{
                'yearly':{},
                     'monthly':{},
                     'stat':{},
                     }                
                }

pickle_out(BASIN)

# 2.3 Create NetCDF files

In [ ]:
#Get inputs for create_NC
cutline=BASIN['geo_data']['basin']
Dir_out=os.path.join(BASIN['Dir'],'data','nc')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)    
template=glob.glob(os.path.join(BASIN['input_data']['yearly']['lcc'][0],'*.tif'))[0]

In [ ]:
### yearly maps
for key in BASIN['input_data']['yearly']:             
    if BASIN['input_data']['yearly'][key] is not None:
        nc_fn=os.path.join(Dir_out,key+'_yearly.nc')        
        dataset={key:[BASIN['input_data']['yearly'][key][0],
                       ('time','latitude', 'longitude'), 
                       {'units': BASIN['input_data']['yearly'][key][1],                                 
                        'quantity':BASIN['input_data']['yearly'][key][2],
                        'source': 'WaPOR', 'period':'year'}]}
        succes=make_netcdf(nc_fn,BASIN['Name'],dataset,template,cutline,step='year')
        if succes:
            BASIN['main_data']['yearly'][key]=nc_fn
            print('Finished {0}_yearly.nc'.format(key))

### monthly maps  
for key in BASIN['input_data']['monthly']:           
    if BASIN['input_data']['monthly'][key] is not None:
        nc_fn=os.path.join(Dir_out,key+'_monthly.nc')                
        dataset={key:[BASIN['input_data']['monthly'][key][0],
                       ('time','latitude', 'longitude'), 
                       {'units': BASIN['input_data']['monthly'][key][1],                                 
                        'quantity':BASIN['input_data']['monthly'][key][2],
                        'source': 'WaPOR', 'period':'month'}]}
        succes=make_netcdf(nc_fn,BASIN['Name'],dataset,template,cutline,step='month')
        if succes:
            BASIN['main_data']['monthly'][key]=nc_fn
            print('Finished {0}_monthly.nc'.format(key))
  
### static maps
for key in BASIN['input_data']['stat']:            
    if BASIN['input_data']['stat'][key] is not None:  
        nc_fn=os.path.join(Dir_out,key+'_stat.nc')                
        dataset={key:[BASIN['input_data']['stat'][key][0],
                       ('latitude', 'longitude'), 
                       {'units': BASIN['input_data']['stat'][key][1],                                 
                        'quantity':BASIN['input_data']['stat'][key][2],
                        'source': 'WaPOR', 'period':'stat'}]}
        succes=make_netcdf(nc_fn,BASIN['Name'],dataset,template,cutline)
        if succes:
            BASIN['main_data']['stat'][key]=nc_fn
            print('Finished {0}stat.nc'.format(key))

pickle_out(BASIN)